# Praca domowa nr 1
## Jan Smoleń, 08-03-2021
Tematem poniższej pracy jest eksploracja zbioru danych dotyczącego pożarów lasów w parku Montesinho w północno-wschodnim rejonie Portugalii.

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas_profiling

In [ ]:
df = pd.read_csv("forest_fires_dataset.csv")
df.head()

In [ ]:
df.describe()

##  Kolumny
**X** - Współrzędne przestrzenne osi X na mapie parku Montesinho: od 1 do 9. </p> 
**Y** - Współrzędne przestrzenne osi Y na mapie parku Montesinho: od 2 do 9.</p>  
**month** - miesiąc </p>  
**day** - dzień tygodnia </p>  
**FFMC** - reprezentuje suchość paliwa ze najmniejszej ściółki leśnej w cieniu okapu lasu. Przyjmuje wartości od 0-101.</p>  
**DMC** - reprezentuje suchość paliwa rozłożonego materiału organicznego pod ściółką, do głębokośći 10cm. Przyjmuje dowolne dodatnie wartości. </p>  
**DC** - reprezentuje suchość dużego materiału organicznego na głębokości większej niż 10 cm. Przyjmuje wartości od 0 do 1000. </p>   
**ISI** - 
Szacuje potencjał rozprzestrzeniania się ognia, wynika z FFMC oraz prędkości wiatru. Przyjmuje dowolne dodatnie wartości. </p>   
**temp** - temperatura w stopniach Celsjusza </p>   
**RH** - względna wilgotność w procentach</p>   
**wind** - prędkość wiatru w km/h.</p>   
**rain** - opady deszczu w mm/m2.</p>   
**area** - spalony obszar lasu w hektarach.</p>   

Użyjemy teraz narzędzia do automatycznej eksploracji danych pandas-profiling żeby poznać trochę bardziej szczegółowe informacje na temat naszych danych.

In [ ]:
df.profile_report()

## Wnioski/pytania
1. Nie występuje problem braku danych </p>  
2. Co oznacza, że spalony obszar wynosi zero? Czy to, że pożar się nie rozprzestrzenił czy może to, że nie pożar nie wystąpił wcale? Jeżeli ta druga opcja, to co to decyduje o wpisaniu danego dnia i obszaru do danych? Może pomiary były wykonywane częściej podczas okresów bardziej podatnych na pożary? Wydaje mi się to nie bez znaczenia w kontekście wykorzystania tych danych do przewidywania zmiennej celu.   </p>    
3. Można się pozbyć zduplikowanych rzędów, ponieważ odzwierciedlają wyniki tych samych pomiarów. </p> 
4. Duża ilość zer w kolumnach area i rain powoduje, że przygotowane przez narządzie wykresy z tymi danymi są mało czytelne. </p>
5. Główne ograniczenia tego narzędzia do automatyzowania eksploracji: dłuższy czas generowania, dużo nieinteresujących nas informacji, mało czytelne wykresy.  

Usuniemy zduplikowane rzędy i przygotujemy wygodniejszą wersję heatmapy korelacji

In [ ]:
df_reduced=df.drop_duplicates()
sns.heatmap(df_reduced.corr(), annot=True, annot_kws={'size': 9}, fmt='.2f')

Zgodnie z oczekiwaniami, wskaźniki oznaczające suchosć ściółki na różnych poziomach są dość mocno skorelowane zarówno między sobą, jak i z temperaturą. Po zrobieniu researchu okazuje się także, że wzór na ISI zależy jedynie od wartości FFMC i prędkości wiatru - może należałoby usunąć te kolumnę? 

Spójrzmy teraz na rozkłady zmiennych objaśniających.

In [ ]:
df_reduced.drop(["area"], axis=1).hist(bins = 40, figsize=(18, 12))
plt.show()

Widzimy że zmienne temp, ISI, RH i wind mają rozkłady przypominające rozkłady normalne. W zmiennej rain dominują wartości zerowe.  Uwagę zwracają pojedyncze wartość ISI i FFMC, która zdecydowanie odstają od reszty. Spójrzmy na wykres punktowy tych dwóch zmiennych i zmiennej celu area.

In [ ]:
plt.scatter(df_reduced["ISI"], df_reduced["area"])

In [ ]:
plt.scatter(df_reduced["FFMC"], df_reduced["area"])

Ponieważ te dwa pomiary znacząco odstają od reszty i nie wpisują się szczególnie w jakiś widoczny trend, usuniemy je z naszych danych. Możemy też się upewnić, że nie należą do tego samego rzędu - tak nie jest. 

In [ ]:
df_reduced=df_reduced.drop([df_reduced["ISI"].idxmax()])
df_reduced=df_reduced.drop([df_reduced["FFMC"].idxmin()])

Przyjrzymy sie jeszcze trochę dokładniej rozkładowi zmiennej celu area.

In [ ]:
sns.displot(df_reduced["area"], bins=40)

Spójrzmy jeszcze, jak rozkładają się wielkości spalonego obszaru w sytuacji, w której wiemy już, że pożar w ogóle wystąpił/rozprzestrzenił się.

In [ ]:
df_drop0=df_reduced[df_reduced['area']!=0]
sns.displot(df_drop0["area"], bins=40)

Zatem widzimy, że w przeważającej liczbie pomiarów pożar albo nie występuje wcale, albo występuje na małym obszarze. Rozkład jest bardzo przekrzywiony w lewo, być może na dalszym etapie do jego modelowania przydatna byłaby transformacja logarytmiczna. Dwie wartości bardzo odstają od reszty, ale moim zdaniem świadczą raczej o pożarach, które wymknęły się spod kontroli niż o błędach w pomiarze, więc je pozostawmy.

Biorąc pod uwagę, że wartości X i Y oznaczają położenie na mapie parku i raczej należałoby je rozpatrywać razem, spróbujemy odtworzyć te mapę i pokazać na niej sumę spalonego obszaru w danych sektorach. Dzięki temu też zobaczymy, jakie obszary w ogóle występują w parku.

In [ ]:
mapa=df_reduced.groupby(["X", "Y"]).agg({'area':'sum'}).reset_index()
mapa = mapa.pivot(index='Y', columns='X', values='area')
mapa=mapa.iloc[::-1]
sns.heatmap(mapa, annot=True, annot_kws={'size': 9}, fmt='.1f')

Teraz zbadamy ilość spalonego obszaru w poszczególnych miesiącach i dniach tygodnia. Ze względu na bardzo liczne zera występujące w kolumnie area oraz nieznany nam sposób wyboru dni i obszarów zawartych w danych, posłużymy się wykresami słupkowymi pokazyjącymi łączną sumę spalonych obszarów.

In [ ]:
tmp_month=df_reduced.groupby(["month"]).agg({'area':'sum'}).reset_index()
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
          "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
tmp_month["month"]=tmp_month["month"].str.capitalize()
tmp_month['month'] = pd.Categorical(tmp_month['month'], categories=months, ordered=True)
tmp_month = tmp_month.sort_values(by="month")
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(tmp_month["month"], tmp_month["area"])


In [ ]:
tmp_day=df_reduced.groupby(["day"]).agg({'area':'sum'}).reset_index()
days=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
tmp_day["day"]=tmp_day["day"].str.capitalize()
tmp_day['day'] = pd.Categorical(tmp_day['day'], categories=days, ordered=True)
tmp_day = tmp_day.sort_values(by="day")
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(tmp_day["day"], tmp_day["area"])

Jak widzimy, znacznie więcej powierzchni uległo pożarom w sierpniu i wrześniu niż w innych miesiącach. Co ciekawe, sobota wyróżnia się od pozostałych dni tygonia - być może zwiększona liczba spacerowiczów ma wpływ na więcej pożarów.